In [1]:
import pandas as pd
import numpy as np
import numba as nb
import os
import concurrent.futures

@nb.jit(nopython=True)
def msk_in_r(coor, halo_center, r):
    n = coor.shape[1]
    where = np.empty(coor.shape[0], dtype=np.bool_)
    for i in range(coor.shape[0]):
        d2 = 0.0
        for j in range(n):
            d2 += (coor[i,j] - halo_center[j])**2
        where[i] = d2 < r**2
    return where
@nb.jit(nopython=True)
def msk_in_radii(coor, halo_center, r1, r2):
    n = coor.shape[1]
    where = np.empty(coor.shape[0], dtype=np.bool_)
    for i in range(coor.shape[0]):
        d2 = 0.0
        for j in range(n):
            d2 += (coor[i,j] - halo_center[j])**2
        where[i] = (d2 < r2**2) & (d2 >= r1**2)
    return where

@nb.jit(nopython=True)
def msk_in_cylinder(coor, halo_center, r1,r2, z): # r, z in cMpc
    n = 2
    where = np.empty(coor.shape[0], dtype=np.bool_)
    for i in range(coor.shape[0]):
        d2 = 0.0
        for j in range(n):
            d2 += (coor[i,j] - halo_center[j])**2
        where[i] = (d2 <= r2**2) & (d2 >= r1**2) & (coor[i,2] < (halo_center[2]+z/2)) & (coor[i,2] >= (halo_center[2]-z/2))
    return where
   

for mf in [15.0]:#[13.0, 13.5, 14.0, 14.5, 15.0]:
    olddatapath = f'/cosma8/data/dp004/dc-chen3/work/bin/halo-radial-profile-in-snapshot/results/results_wrong_wholeboxz_sb/xraysb_csvs_230504_{mf}_groups_1028halos'
    workpath = f'/cosma8/data/dp004/dc-chen3/work/bin/halo-radial-profile-in-snapshot/results/results_add_xraylum_sb_230509/xraysb_csvs_{mf}_groups_1028halos'
    savepath = f'{workpath}/xraylum_csvs_230511_{mf}_groups_radial_pkpc_cylinder'
    os.makedirs(savepath, exist_ok = True)
    df_halo = pd.read_csv(f'{olddatapath}/xray_linelum_snapshot75_halomass_btw_{int(mf*10)}_{int((mf+0.5)*10)}_230404.csv')
    haloids = df_halo['halo_ids']
    halo_centers = np.array([df_halo['x_gasmass_center'], df_halo['y_gasmass_center'], df_halo['z_gasmass_center']]).T
    halo_r200cs = df_halo['r200c']
    xbins = np.linspace(-2,3.1,50)
    props = [('o7f', 'sum'), ('o8', 'sum'), ('fe17', 'sum')]
    def cal_xraylum(k):
        haloid = haloids[k]
        print(haloid)
        halo_cen = halo_centers[k]
        bins = np.power(10, xbins) * 1 #Mpc
        prop_arr  = np.zeros(len(bins))
        newdf_part = pd.read_csv(f'{workpath}/xray_linelum_snapshot75_halo{int(haloid-1)}_partlum.csv')
        df_part = pd.read_csv(f'{olddatapath}/xray_linelum_snapshot75_halo{int(haloid-1)}_partlum_230404.csv')
        right_lumdist = 4*np.pi*((df_part['part_zcoords']-(halo_cen[2]-3.125)) * (1+0.1))**2
        part_flux = newdf_part[f'lum_{prop[0]}']/right_lumdist
        print((df_part['part_zcoords']-(halo_cen[2]-3.125)))
        # print(np.sum(newdf_part[f'lum_{prop[0]}']>0))
        # print(right_lumdist)
        # print(np.sum(part_flux>0))
        for j in range(len(bins)-1):
            radmsk = msk_in_cylinder(np.array([df_part['part_xcoords'], df_part['part_ycoords'], df_part['part_zcoords']]).T, halo_cen, bins[j], bins[j+1], 6.25)
            prop_arr[j] = np.nansum(part_flux[np.array(df_part['jointmsk']) & radmsk])
        print(np.sum(prop_arr>0))
        return prop_arr

    for prop in props:
        output = np.zeros((len(xbins), len(haloids)))
        k = 0
        # with concurrent.futures.ProcessPoolExecutor(8) as executor:
        #     for k, result in enumerate(executor.map(cal_xraylum,np.arange(len(haloids)))):
                # output[:,k] = result
        output[:,k] = cal_xraylum(k)
        df = pd.DataFrame.from_dict(output)
        df.to_csv(f'{savepath}/{prop[0]}.csv')

SyntaxError: invalid syntax (<ipython-input-1-1fc49971f365>, line 41)